In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh, block_diag
import math 
from itertools import combinations

from tabulate import tabulate

In [2]:
def spinless_sub_dim(N,r) : 
    return math.comb(N,r)

def spinless_fock_dim(N) : 
    dim_dict = {} 
    for r in range(N+1) : 
        dim_dict[r] = spinless_sub_dim(N,r)
    return sum(dim_dict.values()), dim_dict

def spinless_basis(N,r) : 
    basis_set = []
    lattice = list(range(N))
    places = list(combinations(lattice, r))
    for combination in places : 
        basis = [False] *N
        for index in combination : 
            basis[index] = True 
        basis_set.append(basis)
    return basis_set
def spinless_states_index(N) : 
    '''Input : N, uses spinless_basis function
    Output : Returns binary of basis states in fermionic Hamiltonian ''' 
    index = []
    for r in range(N+1) : 
        b_set = spinless_basis(N,r)
        new_bset = []
        for b in b_set : 
            new_b = sum([2**(N-i-1)*b[i] for i in range(N)])
            index.append(new_b)
    return index

def tb0_model(N,r,e,t): 
    ''' Generalised Tight Binding (spinless) model for periodic boundary condition
    Input : Number of lattice sites (int), number of electrons(int), hopping constant (int/float), onsite energies (list)
    Output : Tight binding Hamiltonian, eigenvalues and eigenvectors of the matrix ''' 
    dim = spinless_sub_dim(N,r)
    if dim==1 : 
        H = np.ones(1)*r
        eigval = 1*r
        new_vec = [[1]]
    elif r==N : 
        H = [[r]]
        eigval = H[0]
        new_vec = H
    else : 
        H = np.zeros((dim, dim))
        basis_set = spinless_basis(N,r)
        n_diag = np.zeros(dim)
        for i in range(dim) : 
            for j in range(N) : 
                n_diag[i] += e[j]*basis_set[i][j]

        np.fill_diagonal(H,n_diag)
        for basis_index,basis in enumerate(basis_set) : 
            for site in range(len(basis)) : 
                if basis[site] == False and basis[(site+1)%N] == True : 
                    new_state = basis.copy()
                    new_state[site] = True
                    new_state[(site+1)%N] = False 
                    for i in range(len(basis_set)) : 
                        if basis_set[i] == new_state: 
                            f_index = i
                    H[f_index][basis_index] +=t
                if N != 2 : 
                    if basis[site] == True and basis[(site+1)%N] == False : 
                        new_state = basis.copy()
                        new_state[site] = False
                        new_state[(site+1)%N] = True 
                        for i in range(len(basis_set)) : 
                            if basis_set[i] == new_state : 
                                f_index = i
                        H[f_index][basis_index] +=t 
        H[0][0]=1
        eigval,eigvec = np.linalg.eigh(H)
        new_vec = list(zip(*eigvec))
                
    return H,eigval,new_vec

def tb0_full(N,e,t): 
    '''Full Block Diagonal Hamiltonian for some N length closed lattice '''
    H_sub_list = []
    for r in range(N+1) : 
        H_sub = tb0_model(N,r,e,t)[0]
        H_sub_list.append(H_sub)
    H = block_diag(*H_sub_list) 
    return H

def spinless_state_vec(basis) : 
    N = len(basis) 
    r = np.count_nonzero(basis)
    dim = spinless_sub_dim(N,r)
    basis_set = spinless_basis(N,r)
    i =-1
    index=0
    for state in basis_set : 
        i+=1
        vec = np.zeros(dim)
        if state == basis : 
            index = i

        vec[index] = 1
    return vec
    
def density_of_states(eigval,n_centers) :
    gamma = (eigval[-1]-eigval[0])/n_centers
    centers = np.linspace(eigval[0]-n_centers/10, eigval[-1]+n_centers/10, num=n_centers)
    
    density = np.zeros(n_centers)
    for i, center in enumerate(centers):
        if center < eigval[0] or center > eigval[-1]: 
            density[i] = 0 
        else : 
            lorentz = np.sum(1 / np.pi * (gamma / 2) / ((center - eigval)**2 + (gamma / 2)**2))
            density[i] = lorentz
    norm_density = [float(i)/sum(density) for i in density]
    plt.plot(centers,norm_density)
    plt.xlabel('Energy of system')
    plt.ylabel('Density of states')
    plt.title("Density of states")
    return density
     

def state_occupation(states) :
    '''Describes occupation of eigenstates over the lattice sites 
    Input : list of eigenstates
    Output : plots the occuptation number graph and 
    returns list of occupation number over sites for all thes states (list of lists)'''
    num_states = len(states)
    dim = len(states[0])
    vec_site_occ = []
    basis_set = spinless_basis(N,r)
    for index,vec in enumerate(states) : 
        site_occ = np.zeros(N)
        for site in range(N) : 
            for i in range(dim) : 
                site_occ[site] += np.abs(vec[i])**2 *basis_set[i][site]
        plt.plot(range(N), site_occ, label='vec {:.2f}'.format(index))
        vec_site_occ.append(site_occ)
    plt.xlabel('sites')
    plt.ylabel('<n>')
    plt.title("Occupation of states wrt sites")
    plt.legend()

def ferm_mat(mat) : 
    '''Input : SparsePauliOp JW matrix, uses function spinless_states_index
    Output : Returns JW matrix in order of fermionic basis states'''
    class_index = spinless_states_index(N)
    #JW_mat = mat.to_matrix()
    new_mat = np.zeros((2**N,2**N))
    for i in range(2**N) : 
        for j in range(2**N) : 
            new_mat[i][j] = mat[class_index[i]][class_index[j]]
    return new_mat

In [13]:
N =5
r=2
dim = spinless_sub_dim(N,r)
e=[1]*N

t = 7
#e = np.array(range(N))

# e[N//4] = -40
# e[3*N//4] = -40

# for i in range(int(dim/2)) : 
#     e[2*i] = 3

# delta = 10
# e = [random.uniform(-delta,delta) for _ in range(N)]

H,eig,vec= tb0_model(N,r,e,t)
basis_set = spinless_basis(N,r)

#basis = basis_set[2]


In [11]:
# eigval_jw, eigvec_jw = np.linalg.eig()

In [14]:
sub_eigval = []
sub_eigvec = []
for r in range(N+1) : 
    sub_eigval.append(tb0_model(N,r,e,t)[1])
    sub_eigvec.append(tb0_model(N,r,e,t)[2])
full_eigval, full_eigvec = np.linalg.eigh(tb0_full(N,e,t))

In [15]:
sub_eigval

[0,
 array([-10.32623792, -10.32623792,   5.32623792,   5.32623792,
         15.        ]),
 array([-16.44263905, -16.32623792,  -6.8068687 ,  -0.96602404,
         -0.67376208,   8.61598322,   9.        ,   9.        ,
          9.        ,  24.59954857]),
 array([-15.57180027, -15.32623792,  -5.98200754,  -0.26079139,
          0.32623792,   9.26363791,  10.        ,  10.        ,
         10.        ,  25.55096129]),
 array([-8.64957743, -7.32623792,  7.15835976,  8.32623792, 17.49121767]),
 5]

In [12]:
full_eigval

array([-6.        , -6.        , -5.67707825, -5.        ,  0.        ,
        3.        , 15.        , 15.67707825])

In [13]:
for sub_vec in sub_eigvec : 
    for vec in sub_vec : 
        print(vec)
        print('-----------------')

[1]
-----------------
(0.0, -0.7071067811865476, 0.7071067811865475)
-----------------
(-0.8164965809277261, 0.40824829046386313, 0.408248290463863)
-----------------
(-0.5773502691896257, -0.5773502691896261, -0.577350269189626)
-----------------
(-0.8290459349857152, 0.3954003273162917, 0.39540032731629166)
-----------------
(0.0, -0.7071067811865476, 0.7071067811865475)
-----------------
(-0.5591805054574606, -0.5862240025435409, -0.5862240025435408)
-----------------
[1]
-----------------


In [14]:
print(full_eigval)
new_vec = list(zip(*full_eigvec))
for vec in new_vec : 
    #new_list = [x for x in vec if x != 0]
    print(list(vec))
    print('-------------------------')
    


[-6.         -6.         -5.67707825 -5.          0.          3.
 15.         15.67707825]
[0.0, 0.0, -0.7071067811865476, 0.7071067811865475, 0.0, 0.0, 0.0, 0.0]
-------------------------
[-0.0, -0.8164965809277261, 0.40824829046386313, 0.408248290463863, -0.0, -0.0, -0.0, -0.0]
-------------------------
[-0.0, -0.0, -0.0, -0.0, -0.8290459349857152, 0.3954003273162917, 0.39540032731629166, -0.0]
-------------------------
[0.0, 0.0, 0.0, 0.0, 0.0, -0.7071067811865476, 0.7071067811865475, 0.0]
-------------------------
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
-------------------------
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
-------------------------
[0.0, -0.5773502691896257, -0.5773502691896261, -0.577350269189626, 0.0, 0.0, 0.0, 0.0]
-------------------------
[0.0, 0.0, 0.0, 0.0, -0.5591805054574606, -0.5862240025435409, -0.5862240025435408, 0.0]
-------------------------


In [11]:
for r in range(N+1) : 
    print(spinless_basis(N,r))

[[False, False, False, False, False, False, False, False, False, False]]
[[True, False, False, False, False, False, False, False, False, False], [False, True, False, False, False, False, False, False, False, False], [False, False, True, False, False, False, False, False, False, False], [False, False, False, True, False, False, False, False, False, False], [False, False, False, False, True, False, False, False, False, False], [False, False, False, False, False, True, False, False, False, False], [False, False, False, False, False, False, True, False, False, False], [False, False, False, False, False, False, False, True, False, False], [False, False, False, False, False, False, False, False, True, False], [False, False, False, False, False, False, False, False, False, True]]
[[True, True, False, False, False, False, False, False, False, False], [True, False, True, False, False, False, False, False, False, False], [True, False, False, True, False, False, False, False, False, False], [True

In [12]:
tb0_full(N,e,t)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  7., ...,  0.,  0.,  0.],
       [ 0.,  7.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  9.,  7.,  0.],
       [ 0.,  0.,  0., ...,  7.,  9.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., 10.]])

In [13]:
r=2
sum([math.comb(N,k) for k in range(r)])

11

In [14]:
print(tabulate(tb0_full(N,e,t),tablefmt='plain'))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
tb0_model(N,r,t,e)

TypeError: 'int' object is not subscriptable

In [ ]:
state_occupation([vec[0],vec[0]])


In [ ]:
plt.plot(range(N),e)
plt.xlabel('sites')
plt.ylabel('e')


In [ ]:
H

In [ ]:
N=4
for r in range(N+1) : 
    b_set = spinless_basis(N,r)
    new_bset = []
    for b in b_set : 
        new_b = sum([2**(N-i-1)*b[i] for i in range(N)])
        new_bset.append(new_b)
    print(new_bset)

In [ ]:
N=3
from scipy.linalg import block_diag
n = []
for r in range(N+1) : 
    n.append(tb0_model(N,r,e,t)[0])
print(tabulate(block_diag(n[0],n[1],n[2],n[3]),tablefmt='plain'))

In [ ]:
def spinless_state_occupation(states) :
    '''Describes occupation of eigenstates over the lattice sites 
    Input : list of eigenstates
    Output : plots the occuptation number graph and 
    returns list of occupation number over sites for all thes states (list of lists)'''
    num_states = len(states)
    dim = len(states[0])
    states_occ = []
    for index in range(num_states) : 
        mod_sq = np.zeros(dim)
        for i in range(dim) : 
            mod_sq[i] = np.abs(states[index][i])**2
        states_occ.append(mod_sq)

        plt.plot(range(dim), mod_sq, label='vec {:.2f}'.format(index))
    plt.xlabel('sites')
    plt.ylabel('<n>')
    plt.title("Occupation of states wrt sites")
    plt.legend()

    return states_occ

def density_of_states(eigval,n_centers) : 
    '''
    Input : eigenvalues of the model, number of points where density is to be found
    Output : density of states, plots density of states graph 
    '''
    gamma = (eigval[-1]-eigval[0])/n_centers
    centers = np.linspace(eigval[0]-n_centers/10, eigval[-1]+n_centers/10, num=n_centers)
    
    density = np.zeros(n_centers)
    for i, center in enumerate(centers):
        if center < eigval[0] or center > eigval[-1]: 
            density[i] = 0 
        else : 
            lorentz = np.sum(1 / np.pi * (gamma / 2) / ((center - eigval)**2 + (gamma / 2)**2))
            density[i] = lorentz
    norm_density = [float(i)/sum(density) for i in density]
    plt.plot(centers,norm_density)
    plt.xlabel('Energy of system')
    plt.ylabel('Density of states')
    plt.title("Density of states")
    return density

In [ ]:
spinless_state_occupation([vec[0],vec[0]])
